In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import urllib
import time
import pandas as pd
import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tensorflow.keras.preprocessing.image import img_to_array
from skimage.color import rgba2rgb

In [27]:
#activating firefox
driver = webdriver.Firefox()

In [28]:
#setting drive to wait until element apparing
driver.implicitly_wait(10)

In [29]:
driver.get('https://www.ebay-kleinanzeigen.de/s-notebooks/anbieter:privat/anzeige:angebote/preis:100:/seite:10/c278')

In [ ]:
#scraping image link and descriptin from website
src = []
main = []
price = []
desc = []


for j in range(20,48):
    path = f'https://www.ebay-kleinanzeigen.de/s-handy-telekom/lg_handy/anbieter:privat/anzeige:angebote/seite:{j}/c173+handy_telekom.art_s:lg_handy+handy_telekom.device_equipment_s:only_device'
    driver.get(path)
    for i in range(1,26):
        try:
            if i < 10: 
                time.sleep(2)
                driver.execute_script("window.scrollBy(0, 500)")
                time.sleep(2)
            elif i < 20:
                time.sleep(2)
                driver.execute_script("window.scrollBy(0, 1000)")
                time.sleep(2)
            elif i < 30:
                time.sleep(2)
                driver.execute_script("window.scrollBy(0, 500)")
                driver.execute_script("window.scrollBy(0, 500)")
                driver.execute_script("window.scrollBy(0, 500)")
                time.sleep(2)
            x = f'/html/body/div[1]/div[2]/div/div[3]/div[2]/div[2]/div[2]/ul/li[{i}]/article/div[1]/a/div/img'
            a = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, x)))
            s = a.get_attribute('src')
            x = f'/html/body/div[1]/div[2]/div/div[3]/div[2]/div[2]/div[2]/ul/li[{i}]/article/div[2]/div[2]/h2/a'
            a = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, x)))
            m = a.text
            x = f'/html/body/div[1]/div[2]/div/div[3]/div[2]/div[2]/div[2]/ul/li[{i}]/article/div[2]/div[2]/p[1]'
            a = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, x)))
            d = a.text
            x = f'/html/body/div[1]/div[2]/div/div[3]/div[2]/div[2]/div[2]/ul/li[{i}]/article/div[2]/div[2]/p[2]'
            a = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, x)))
            p = a.text
            
    
            
            if s != "" and m != "" and p != "" and d != "" and (s not in sr) and (s not in src):
                src.append(s)
                main.append(m)
                price.append(p)
                desc.append(d)
                if len(src) > 1 and len(src) < 3:
                    print(pd.DataFrame(zip(src,main,price,desc)).tail(2))
            
            print("try",j,i)
        except:
            driver.get(path)
            time.sleep(2)
            driver.execute_script("window.scrollBy(0, 1000)")
            driver.execute_script("window.scrollBy(0, 1000)")
            time.sleep(2)
            print("cont",j,i)
        print(len(src))              
        
    if len(src) > 50:
        break

In [42]:
src[0]

'https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/5dIAAOSwux1hRI9j/$_2.JPG'

In [6]:
#making data frame based on phone labes
df = pd.DataFrame(zip(src,main,desc,price,['apple' for i in range(len(src))])).drop_duplicates()
df.columns = ['link','main','dec','price','marka'] 

In [8]:
df_ = pd.DataFrame(zip(src,main,desc,price,['samsung' for i in range(len(src))])).drop_duplicates()
df_.columns = ['link','main','dec','price','marka'] 

In [12]:
df_n = pd.DataFrame(zip(src,main,desc,price,['nokia' for i in range(len(src))])).drop_duplicates()
df_n.columns = ['link','main','dec','price','marka'] 

In [31]:
df_h = pd.DataFrame(zip(src,main,desc,price,['htc' for i in range(len(src))])).drop_duplicates()
df_h.columns = ['link','main','dec','price','marka'] 

In [35]:
df_l = pd.DataFrame(zip(src,main,desc,price,['lg' for i in range(len(src))])).drop_duplicates()
df_l.columns = ['link','main','dec','price','marka'] 

In [36]:
df_t = pd.concat([df,df_, df_n,df_h,df_l]).reset_index().drop('index',1)

In [118]:
#making array to save images
img_array = []


for i in range(df_t):
    path = df_t.link[i]
    driver.get(path)
    x = '/html/body/img'
    a = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, x)))
    with open(f'name0', 'wb') as file:
        file.write(a.screenshot_as_png)
    time.sleep(1)
    im = Image.open("name0")
    X = img_to_array(im)
    img = im.resize((128,128), Image.ANTIALIAS)
    img_array.append(rgba2rgb(img_to_array(img)/255))
    print(i)

np.save('mod.npy', np.array(img_array))